 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"></ul></div>

In [1]:
import json
import time

from __future__ import print_function # For py 2.7 compat

from IPython.html import widgets # Widget definitions
from IPython.display import display # Used to display widgets in the notebook
from IPython.utils.traitlets import Unicode # Used to declare attributes of our widget

C:\Users\Pawan\Anaconda3\lib\site-packages\IPython\html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
C:\Users\Pawan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  


In [2]:
%%javascript

require(["widgets/js/widget"], function(WidgetManager){
    
    var GraphView = IPython.DOMWidgetView.extend({
        render: function(){
            // alert('render');
            this.$graph = $('<iframe id="plotly-graph-1" src="https://plot.ly/~chris/3375" seamless width="600" height=400></iframe>')
                          .appendTo(this.$el);
            this.$loading = $('<div id="graph-loading">Initializing...</div>')
                            .appendTo(this.$el);
            
            var pinger = setInterval(function(){
                var plot = $('#plotly-graph-1')[0].contentWindow;                
                console.log('posting ping');
                plot.postMessage({ping: true}, 'https://plot.ly');
            }, 2000);   
            
            window.addEventListener('message', function(e) {
                var message = e.data;
                if(message==='pong') {
                    console.log('pong received, ready to receive');
                    $('#graph-loading').hide()
                    clearInterval(pinger);
                }
            });
        },
        
        update: function() {
            var message = this.model.get('value');
            // alert(message)
            
            message = JSON.parse(message);
            
            var plot = $('#plotly-graph-1')[0].contentWindow;
            plot.postMessage(message, 'https://plot.ly');
            
            // this.$graph.val(this.model.get('value'));
            return GraphView.__super__.update.apply(this);
        },
    });
    
    // Register the DatePickerView with the widget manager.
    WidgetManager.register_widget_view('GraphView', GraphView);
});

<IPython.core.display.Javascript object>

In [3]:
class GraphWidget(widgets.DOMWidget):
    _view_name = Unicode('GraphView', sync=True)
    value = Unicode(sync=True)
    description = Unicode(sync=True)

C:\Users\Pawan\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: metadata {'sync': True} was set from the constructor. With traitlets 4.1, metadata should be set using the .tag() method, e.g., Int().tag(key1='value1', key2='value2')
  


In [4]:
g = GraphWidget()
g

GraphWidget()

In [5]:
def restyle(data, traces=None):
    
    message = {'restyle': data}
    if traces:
        message['traces'] = traces
        # message['restyle']['traces'] = traces
    
    g.value = json.dumps(message)

In [6]:
# Update the graph's data
restyle({'x': [[1,2,3], 
               [1, 2, 4]], 
         'y': [[10,20,30],
               [5, 8, 14]]
}, traces=[0,1])

In [7]:
# Clear the graph
restyle({'x': [], 'y': []})

In [8]:
# Update the graph in real-time
x = []
y = []
for i in range(500):
    x.append(i)
    y.append(i)
    
    restyle({'x': [y], 'y': [x]}, traces=[0,1])
    time.sleep(100./1000.)